<a href="https://colab.research.google.com/github/Zenith1618/Web-Scraping/blob/main/Dataset_Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install MechanicalSoup

In [2]:
import requests
import pandas as pd
import mechanicalsoup

In [3]:
browser = mechanicalsoup.Browser()

In [4]:
url = 'https://report.grid-india.in/psp_report.php'
page = browser.get(url, verify=False)

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'report.grid-india.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [5]:
input_date = "2022-08-14"  # Replace with the desired date
form = page.soup.find("form", class_="date-form")
date_input = form.find("input", type="date")
date_input["value"] = input_date

In [6]:
response = browser.submit(form, page.url, verify=False)

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'report.grid-india.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [16]:
print(response.text)

<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>File Browser</title>
<style>
    body {
        font-family: Arial, sans-serif;
        margin: 0;
        padding: 0;
        background-color: #f8f9fa;
        display: flex;
        justify-content: center;
        align-items: center;
    }

    .form {
        flex: 1;
        padding: 30px;
        border-right: 1px solid #ccc;
    }

    .content {
        flex: 3;
        padding: 30px;
    }

    .pdf-container {
        width: 100%;
        height: 600px;
    }

    .pdf-frame {
        width: 100%;
        height: 600px;
        border: none;
    }

    h6 {
        margin-top: 0;
        color: #343a40;
        margin-bottom: 30px;
        text-align: center;
        font-weight: bold;
    }

    .date-form {
        display: flex;
        justify-content: center;
        margin-bottom: 30px;
    }

    .date-form input[type="date"] {
    

In [17]:
from bs4 import BeautifulSoup

In [18]:
soup = BeautifulSoup(response.text, "html.parser")
pdf_links = soup.find_all("a", href=True)
pdf_link = None

In [19]:
for link in pdf_links:
    if link["href"].endswith(".pdf"):
        pdf_link = link["href"]
        print("Found PDF link:", pdf_link)
        break

if not pdf_link:
    print("No PDF link found.")
    exit()

Found PDF link: ./ReportData/Daily Report/PSP Report/2022-2023/August 2022/14.08.22_NLDC_PSP.pdf


In [23]:
from urllib.parse import urljoin

In [25]:
# Step 6: Download the PDF file
base_link = "https://report.grid-india.in/"
down_link = urljoin(base_link, pdf_link)
pdf_response = requests.get(down_link, verify=False)
pdf_filename = "temp.pdf"
with open(pdf_filename, "wb") as pdf_file:
    pdf_file.write(pdf_response.content)

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'report.grid-india.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [15]:
!pip install PyPDF2
import PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.8 MB/s eta 0:00:00


In [27]:
#Use PyPDF2 to extract content from page 2
pdf_filename = "temp.pdf"  # Replace with the actual filename
output_pdf_filename = "page2.pdf"  # Replace with the desired output filename

with open(pdf_filename, "rb") as pdf_file:
    pdf_reader = PyPDF2.PdfReader(pdf_file)
    pdf_writer = PyPDF2.PdfWriter()

    # Add only page 2 to the new PDF writer
    pdf_writer.add_page(pdf_reader.pages[1])

    # Save the new PDF containing only page 2
    with open(output_pdf_filename, "wb") as output_pdf_file:
        pdf_writer.write(output_pdf_file)

print("Page 2 extracted and saved to", output_pdf_filename)

Page 2 extracted and saved to page2.pdf


In [28]:
!pip install tabula-py pandas openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 22.1 MB/s eta 0:00:00


In [29]:
import tabula

In [30]:
pdf_path = "temp.pdf"



In [31]:
tables = tabula.read_pdf(pdf_path, pages="all", multiple_tables=True)

# Assume you want the first table (use indexing if there are multiple tables)
table_data = tables[0]

Aug 15, 2023 7:21:23 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>
Aug 15, 2023 7:21:23 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>
Aug 15, 2023 7:21:23 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Aug 15, 2023 7:21:26 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Aug 15, 2023 7:21:26 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Aug 15, 2023 7:21:26 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Aug 15, 2023 7:21:28 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Aug 15, 2023 7:21:28 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>



In [32]:
import pandas as pd

# Convert the table data into a pandas DataFrame
df = pd.DataFrame(table_data)

# Save the DataFrame into an Excel file
excel_output = "output.xlsx"
df.to_excel(excel_output, index=False, engine="openpyxl")
print("Data extracted and saved to", excel_output)

Data extracted and saved to output.xlsx


In [35]:
# Load the output Excel file
output_excel_file = "output.xlsx"  # Replace with the actual filename
df_output = pd.read_excel(output_excel_file)

# Extract the date and total power columns
date = input_date  # Assuming the date is in the first row, first column
total_power_column = df_output.iloc[0, 6]  # Assuming total power is in the second row, seventh column

# Create a new DataFrame with the extracted data
new_data = pd.DataFrame({"Date": [date], "Total Power": [total_power_column]})

# Print the new dataset
print(new_data)



         Date Total Power
0  2022-08-14      169739
